In [3]:
import os
root_dir = "python"
_, current_folder = os.path.split(os.getcwd())
if current_folder != root_dir:
    os.chdir("../")
    
os.getcwd()

'D:\\Archiv\\Studium\\Master\\6.-Semester\\Masters_Thesis\\Git\\acoustic_covid_detection\\python'

In [2]:
path = os.environ.get("PATH")
min_additional_path = "C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\bin;C:\\Users\\micha\\anaconda3\\envs\\ai38\\Library\\bin;"
combined_path = min_additional_path + path
os.environ["PATH"] = combined_path

In [545]:
import pickle
import pandas as pd
import random
import numpy as np
import collections

In [5]:
with open("data/Coswara_processed/pickles/participant_objects_new.pickle", "rb") as f:
            all_participants = pickle.load(f)
        
with open("data/Coswara_processed/pickles/participants_validLabelsOnly.pickle", "rb") as f:
            only_valid_participants = pickle.load(f)

with open("data/Coswara_processed/pickles/participants_oversampledCovidPositives_validLabelsOnly_augmented.pickle", "rb") as f:
            augmented_participants = pickle.load(f)

In [9]:
metadata = pd.read_csv("data/Coswara_processed/full_meta_data.csv")

In [15]:
len(metadata), len(all_participants), len(only_valid_participants), len(augmented_participants)

(2746, 2703, 2483, 4517)

In [33]:
len(metadata), metadata.covid_label.value_counts()

(2746,
 0.0    1838
 1.0     681
 Name: covid_label, dtype: int64)

In [49]:
cleaned_metadata = metadata[metadata.covid_label.isna() == False]
len(cleaned_metadata), cleaned_metadata.covid_label.value_counts()

(2519,
 0.0    1838
 1.0     681
 Name: covid_label, dtype: int64)

In [52]:
cleaned_metadata2 = cleaned_metadata[cleaned_metadata["duration_original_cough-heavy"] > 0.]
# cleaned_metadata2 = cleaned_metadata[cleaned_metadata.recording_invalid == False]
len(cleaned_metadata2)

2435

In [53]:
cleaned_metadata3 = cleaned_metadata2[cleaned_metadata2.audio_quality_heavy_cough > 0.0]
len(cleaned_metadata3)

1998

In [76]:
def get_list_of_ids_from_participants(parts):
    ids_pos, ids_neg, ids_invalid = [], [], []
    for part in parts:
        if part.get_label() == 1:
            ids_pos.append(part.id)
        elif part.get_label() == 0:
            ids_neg.append(part.id)
        else:
            ids_invalid.append(part.id)
    return ids_neg, ids_pos, ids_invalid

In [77]:
ids_neg, ids_pos, ids_invalid = get_list_of_ids_from_participants(all_participants)
len(ids_neg), len(ids_pos),  len(ids_invalid)

(1805, 678, 220)

In [79]:
ids_neg, ids_pos, ids_invalid = get_list_of_ids_from_participants(augmented_participants)
len(ids_neg), len(ids_pos),  len(ids_invalid)

(1805, 2712, 0)

In [215]:
ids_neg, ids_pos, ids_invalid = get_list_of_ids_from_participants(only_valid_participants)
len(ids_neg), len(ids_pos),  len(ids_invalid)

(1805, 678, 0)

In [459]:
def randomly_split_list_into_two(input_list, ratio=0.8, random_seed=None):
    input_list_temp = input_list.copy()
    print(input_list_temp)
    
    split_index = int(np.floor(len(input_list_temp)*0.8))
    random.Random(random_seed).shuffle(input_list_temp)
    return input_list_temp[:split_index], input_list_temp[split_index:]

In [223]:
ratio=0.8
id_neg_train, id_neg_eval = randomly_split_list_into_two(ids_neg, ratio=ratio, random_seed=1594487326)
id_pos_train, id_pos_eval = randomly_split_list_into_two(ids_pos, ratio=ratio, random_seed=3216549875)
len(id_neg_train), len(id_neg_eval), len(id_pos_train), len(id_pos_eval)

(1444, 361, 542, 136)

In [224]:
train_ids = id_pos_train + id_neg_train
eval_ids = id_pos_eval + id_neg_eval
len(train_ids), len(eval_ids)

(1986, 497)

In [553]:
def get_participant_from_id(parts, user_id):
    for part in parts:
        if part.id == user_id:
            return part
        else:
            print(f"no user for id {user_id} found\n")
            return None

In [244]:
user_id = random.choice(train_ids)
part = get_participant_from_id(all_participants, user_id)

In [248]:
train_split = [get_participant_from_id(all_participants, user_id) for user_id in train_ids]
val_split = [get_participant_from_id(all_participants, user_id) for user_id in eval_ids]

In [251]:
len(train_split), len(val_split)

(1986, 497)

In [252]:
ids_neg, ids_pos, ids_invalid = get_list_of_ids_from_participants(train_split)
len(ids_neg), len(ids_pos), len(ids_invalid)

(1444, 542, 0)

In [253]:
ids_neg, ids_pos, ids_invalid = get_list_of_ids_from_participants(val_split)
len(ids_neg), len(ids_pos), len(ids_invalid)

(361, 136, 0)

In [529]:
augm_ids_neg, augm_ids_pos, augm_ids_invalid = get_list_of_ids_from_participants(augmented_participants)
len(augm_ids_neg), len(augm_ids_pos),  len(augm_ids_invalid)

(1805, 2712, 0)

In [547]:
# check the total amount of positive (augmented) samples VS the amount of unique user_ids within the positive samples
len(augm_ids_pos), len(collections.Counter(augm_ids_pos))

(2712, 678)

In [579]:
augmented_train_split = [part for part in augmented_participants if part.id in train_ids]
augmented_eval_split = [part for part in augmented_participants if part.id in eval_ids]

len(augmented_train_split), len(augmented_eval_split)

(3612, 905)

In [583]:
neg_ids, pos_ids, _ = get_list_of_ids_from_participants(augmented_train_split)
augmented_ids_train = neg_ids + pos_ids
# number of samples in augmented train set vs number of unique samples in train set
print((len(neg_ids), len(pos_ids)))
print((len(collections.Counter(neg_ids)), len(collections.Counter(pos_ids))))

(1444, 2168)
(1444, 542)


In [584]:
neg_ids, pos_ids, _ = get_list_of_ids_from_participants(augmented_eval_split)
augmented_ids_eval = neg_ids + pos_ids

# number of samples in augmented eval set vs number of unique samples in eval set
print((len(neg_ids), len(pos_ids)))
print((len(collections.Counter(neg_ids)), len(collections.Counter(pos_ids))))

(361, 544)
(361, 136)


In [589]:
print(len(augmented_ids_train), len(augmented_ids_eval))
print(len(collections.Counter(augmented_ids_train)), len(collections.Counter(augmented_ids_eval)))

3612 905
1986 497


In [594]:
def are_elements_of_two_lists_overlapping(list1, list2):
    n_overlaps, n_unique = 0, 0
    for element in collections.Counter(list1):
        if element in list2:
            print("overlap detected... fuuuuuck")
            n_overlaps += 1
        else:
            n_unique += 1
    return n_unique, n_overlaps
    

In [595]:
are_elements_of_two_lists_overlapping(augmented_ids_train, augmented_ids_eval)

(1986, 0)

In [596]:
are_elements_of_two_lists_overlapping(augmented_ids_eval, augmented_ids_train)

(497, 0)